In [1]:
# plaidml

import plaidml.keras
plaidml.keras.install_backend()

In [2]:
# Data Loader

import csv
import cv2
import numpy as np
from PIL import Image
from keras.preprocessing import image
from keras.utils import Sequence, to_categorical

def load_csv(folder, csvname):
    # Read CSV Data
    _file = open('{}/{}'.format(folder, csvname), 'r', encoding='utf-8')
    _reader = csv.reader(_file)
    
    filenames = []
    labels = {}
    for line in _reader:
        _x = '{}/{}'.format(folder, line[0])
        _y = [int(it) for it in line[1:]]
        filenames.append(_x)
        labels[_x] = _y
    _file.close()
    
    return filenames, labels

class SVHNLoader(Sequence):
    def __init__(self, filenames, labels, batch_size=32, dim=(48, 48), n_channels=1,
                 shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.filenames = filenames
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(len(self.filenames) / self.batch_size))
    
    def __getitem__(self, index):
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        filenames_temp = [self.filenames[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(filenames_temp)

        return X, y
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.filenames))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def __data_generation(self, filenames_temp):
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size, 2))

        # Generate data
        for idx, filename in enumerate(filenames_temp):
            # Store sample
            X[idx,] = self.__preprocess_image(filename)

            # Store class
            _y = self.labels[filename]
            y[idx] = to_categorical(_y)

        return X, y
    
    def __preprocess_image(self, filename):
        '''
        Image Preprocessing (grayscale)
        '''
        img = image.load_img(filename, target_size=(48, 48))
        img_tensor = image.img_to_array(img)
        img_tensor = np.dot(img_tensor[...,:3], [0.299, 0.587, 0.114])
        img_tensor = np.reshape(img_tensor, (48, 48, 1))
        img_tensor /= 255.0
        img_tensor = img_tensor - img_tensor.mean()
        return img_tensor

In [3]:
# Keras

import keras
import tensorflow as tf
from keras.callbacks import EarlyStopping
from keras.layers import Input, BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Dense, Dropout, Flatten
from keras import optimizers

In [9]:
from keras.applications import VGG16

def designed_model_cnn():
    inp = Input(shape=(48, 48, 1))
    
    x = Conv2D(16, (5, 5), padding='same', activation='relu')(inp)
    x = Conv2D(16, (5, 5), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=2, strides=2)(x)
    x = Dropout(0.15)(x)
    
    x = Conv2D(32, (5, 5), padding='same', activation='relu')(x)
    x = Conv2D(32, (5, 5), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=2, strides=2)(x)
    x = Dropout(0.15)(x)
    
    x = Conv2D(64, (5, 5), padding='same', activation='relu')(x)
    x = Conv2D(64, (5, 5), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=2, strides=2)(x)
    x = Dropout(0.15)(x)
        
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    out = Dense(2, activation='softmax')(x)
    
    model = keras.Model(inputs=inp, outputs=out)
    optim = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001, amsgrad=True)
    model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
    model.summary()
    return model

def designed_model():
    inp = Input(shape=(48, 48, 1))
    x = Flatten()(inp)
    x = Dense(512, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    out = Dense(2, activation='softmax')(x)
    
    model = keras.Model(inputs=inp, outputs=out)
    optim = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001, amsgrad=True)
    model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
    return model

designed_model_cnn().summary()
# designed_model().summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 48, 48, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 16)        416       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 16)        6416      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 32)        12832     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 32)        25632     
__________

In [5]:
filenames = {'train': None, 'test': None}
labels = {'train': None, 'test': None}
filenames['train'], labels['train'] = load_csv('./datasets/checker_train', 'labels.csv')
filenames['test'], labels['test'] = load_csv('./datasets/checker_test', 'labels.csv')

train_dl = SVHNLoader(filenames['train'], labels['train'], batch_size=64)
test_dl = SVHNLoader(filenames['test'], labels['test'], batch_size=64)

In [6]:
# model = vgg_based_model()
model = designed_model()
model.summary()

callbacks = [
    EarlyStopping(monitor='val_loss')
]

# # model1.fit_generator(generator=train_dl,
# #                     validation_data=test_dl,
# #                     use_multiprocessing=True,
# #                     workers=6,
# #                     epochs=25,
# #                     callbacks=callbacks,
# #                     verbose=0)


model.fit_generator(generator=train_dl,
                    validation_data=test_dl,
                    epochs=25,
                    callbacks=callbacks)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 48, 48, 1)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2304)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               1180160   
_________________________________________________________________
dense_7 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_8 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_9 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_10 (Dense)             (None, 2)                 130       
Total para

In [8]:
Save Model
model.save('./checker_model.h5')

from keras.utils.vis_utils import plot_model
plot_model(model, to_file='./model_plot.png', show_shapes=True, show_layer_names=True)

In [28]:
import time

def read_img(filename):
    img = image.load_img(filename, target_size=(48, 48))
    img_tensor = image.img_to_array(img)
    img_tensor = np.dot(img_tensor[...,:3], [0.299, 0.587, 0.114])
    img_tensor = np.reshape(img_tensor, (48, 48, 1))
    img_tensor /= 255.0
    img_tensor = img_tensor - img_tensor.mean()
    return img_tensor


# def predict(filename):
#     img = read_img(filename)
#     dat = model.predict(np.array([img]))
#     return dat

# def predict_with_time(folder, filename):
#     t1 = time.time()
#     output = predict('{}/{}'.format(folder, filename))
#     t2 = time.time()
    
#     print('=' * 12, filename, '=' * 12)
#     print('Raw Output :', output)
#     print('Elapsed Time :', round(t2 - t1, 6))
#     return output
    

# folder = './datasets/checker_test'
# out = predict_with_time(folder, 'k10874.png')
# out[0][0]

(48, 48, 1)